# Investigation into Optimal Badminton Strategy for Men's Singles

Goal:
- Match Win Prediction and most important features
- Point by point and optimal tactic analysis

Areas for future research:
- Individual Player statistics and analysis

In [17]:
from fastai import *
from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import pandas as pd
import numpy as np
from pathlib import Path
import os
import matplotlib as plt

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8


In [6]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [16]:
path = Path("badminton-bwf-world-tour")

if not path.exists():
    os.system(f"kaggle datasets download -d sanderp/badminton-bwf-world-tour -p {path}")

In [ ]:

Path.BASE_PATH = path


 if not path.exists():
        path.mkdir(parents=true)
        api.competition_download_cli(comp, path=path)
        shutil.unpack_archive(str(path/f'{comp}.zip'), str(path))

        


# Things to explore:
# Win%
# Streak strategy, etc

# Service impact